In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [2]:
df = pd.read_csv("raw_ksi_data.csv")

In [3]:
df.shape

(18957, 54)

In [4]:
df.sample(2)

,OBJECTID,INDEX,ACCNUM,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,x,y
14632,14633,80945390,NaN,6/15/2018 8:00:00 AM,1928,5951 STEELES AVE E,NaN,3 m West of,Major Arterial,Scarborough,...,NaN,NaN,NaN,130,Milliken,130,Milliken (130),D42,640126.1108,4.855042e+06
15371,15372,81473536,NaN,2/14/2019 10:00:00 AM,1112,YONGE ST,WILLIAM CARSON CRES,2 m South of,Major Arterial,North York,...,Yes,NaN,NaN,40,St.Andrew-Windfields,40,St.Andrew-Windfields (40),D32,628226.1093,4.845030e+06


In [5]:
df.drop(['ACCNUM','OBJECTID', 'HOOD_140', 'NEIGHBOURHOOD_140', 'OFFSET', 'ROAD_CLASS', 'ACCLOC', 'INVAGE', 'INITDIR'], axis=1, inplace=True)

In [6]:
df.drop(['VEHTYPE', 'MANOEUVER', 'DRIVACT', 'DRIVCOND', 'PEDTYPE', 'PEDACT', 'CYCLISTYPE', 'CYCACT', 'CYCCOND', 'PEDESTRIAN', "TRSN_CITY_VEH"], axis=1, inplace=True)

In [7]:
df.drop(['EMERG_VEH', 'PASSENGER', 'REDLIGHT', 'DISABILITY', 'DIVISION', 'SPEEDING', 'PEDCOND', 'AG_DRIV','x', 'y' ], axis=1, inplace=True)

In [8]:
df.drop(['CYCLIST', 'IMPACTYPE', 'INVTYPE', 'INDEX', 'TRAFFCTL', 'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK', 'ALCOHOL'], axis=1, inplace=True)

In [9]:
df.shape

(18957, 15)

In [10]:
df.sample(2)

,DATE,TIME,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,INJURY,FATAL_NO,HOOD_158,NEIGHBOURHOOD_158
8318,6/5/2012 8:00:00 AM,1807,QUEEN ST W,PETER ST,Toronto and East York,43.649279,-79.393665,Clear,Daylight,Dry,Non-Fatal Injury,NaN,NaN,164,Wellington Place
5637,3/1/2010 10:00:00 AM,1700,KINGSTON RD,SHEPPARD AVE E,Scarborough,43.798911,-79.146220,Clear,Daylight,Dry,Non-Fatal Injury,Major,NaN,133,Centennial Scarborough


In [11]:
df['TIME'] = df['TIME'].astype(str).str.zfill(4)
df['DATE'] = pd.to_datetime(df['DATE'], format='%m/%d/%Y %I:%M:%S %p')

# Extract DOW and HOUR and YEAR
df['DOW'] = df['DATE'].dt.day_name() 
df['HOUR'] = df['TIME'].str[:2].astype(int)
df['YEAR'] = df['DATE'].dt.year
df['MONTH'] = df['DATE'].dt.month

In [12]:
df.sample(3)

,DATE,TIME,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,INJURY,FATAL_NO,HOOD_158,NEIGHBOURHOOD_158,DOW,HOUR,YEAR,MONTH
890,2006-08-20 08:00:00,1543,COLLEGE ST,CLINTON ST,Toronto and East York,43.655345,-79.412990,Clear,Daylight,Dry,Non-Fatal Injury,Minimal,NaN,81,Trinity-Bellwoods,Sunday,15,2006,8
17476,2021-11-21 10:00:00,1923,JANE ST,SHEPPARD AV W,Etobicoke York,43.739379,-79.513187,Rain,"Dark, artificial",Wet,Non-Fatal Injury,Major,NaN,25,Glenfield-Jane Heights,Sunday,19,2021,11
10639,2014-04-24 08:00:00,1956,MAVETY ST,DUNDAS ST W,Toronto and East York,43.665415,-79.465877,Clear,"Dusk, artificial",Dry,Non-Fatal Injury,Major,NaN,90,Junction Area,Thursday,19,2014,4


In [13]:
df["INJURY"].unique()

array(['Major', 'Minor', nan, 'Fatal', 'Minimal'], dtype=object)

In [14]:
df["INJURY"].value_counts()

INJURY
Major      6445
Minor      1479
Minimal    1160
Fatal       976
Name: count, dtype: int64

In [15]:
df['INJURY'] = df['INJURY'].fillna('Minimal')

In [16]:
df["ACCLASS"].unique()

array(['Non-Fatal Injury', 'Fatal', 'Property Damage O', nan],
      dtype=object)

In [17]:
df["ACCLASS"].value_counts()

ACCLASS
Non-Fatal Injury     16268
Fatal                 2670
Property Damage O       18
Name: count, dtype: int64

In [18]:
df['ACCLASS'] = df['ACCLASS'].replace('Property Damage O', 'Non-Fatal Injury')
df['ACCLASS'] = df['ACCLASS'].fillna('Non-Fatal Injury')

In [19]:
df['SEVERE_COLLISION'] = (
    (df['INJURY'].isin(['Major', 'Fatal'])) | 
    (df['ACCLASS'] == 'Fatal')
).astype(int)

In [20]:
df['FATAL_NO'] = df['FATAL_NO'].fillna(0)

In [21]:
df["VISIBILITY"].value_counts()

VISIBILITY
Clear                     16373
Rain                       1976
Snow                        356
Other                        98
Fog, Mist, Smoke, Dust       52
Freezing Rain                47
Drifting Snow                21
Strong wind                  10
Name: count, dtype: int64

In [22]:
df['VISIBILITY'] = df['VISIBILITY'].replace({
    'Other': 'Clear',
    'Freezing Rain': 'Rain',
    'Strong wind': 'Rain',
    'Fog, Mist, Smoke, Dust': 'Rain',
    'Drifting Snow': 'Snow'
})

In [23]:
df["VISIBILITY"].value_counts()

VISIBILITY
Clear    16471
Rain      2085
Snow       377
Name: count, dtype: int64

In [24]:
df["LIGHT"].value_counts()

LIGHT
Daylight                10779
Dark                     3746
Dark, artificial         3552
Dusk                      253
Dusk, artificial          253
Daylight, artificial      151
Dawn                      112
Dawn, artificial          101
Other                       6
Name: count, dtype: int64

In [25]:
df['LIGHT'] = df['LIGHT'].replace({
    'Daylight, artificial': 'Daylight',
    'Dark, artificial': 'Dark',
    'Dusk, artificial': 'Dusk',
    'Dawn, artificial': 'Dawn',
    'Other': 'Daylight'
})

In [26]:
df["LIGHT"].value_counts()

LIGHT
Daylight    10936
Dark         7298
Dusk          506
Dawn          213
Name: count, dtype: int64

In [27]:
df.rename(columns={'RDSFCOND': 'ROAD_CONDITION'}, inplace=True)

In [28]:
df["ROAD_CONDITION"].value_counts()

ROAD_CONDITION
Dry                     15231
Wet                      3140
Loose Snow                174
Other                     147
Slush                     102
Ice                        78
Packed Snow                44
Loose Sand or Gravel       11
Spilled liquid              1
Name: count, dtype: int64

In [29]:
df['ROAD_CONDITION'] = df['ROAD_CONDITION'].replace({
    'Spilled liquid': 'Wet',
    'Loose Snow': 'Snow_Ice',
    'Packed Snow': 'Snow_Ice',
    'Slush': 'Snow_Ice',
    'Ice': 'Snow_Ice',
    'Loose Sand or Gravel': 'Dry',
    'Other': 'Dry'
})

In [30]:
df["ROAD_CONDITION"].value_counts()

ROAD_CONDITION
Dry         15389
Wet          3141
Snow_Ice      398
Name: count, dtype: int64

In [31]:
df['DATE'] = pd.to_datetime(df['DATE']).dt.date

In [32]:
df.drop(['TIME'], axis=1, inplace=True)#since hour col, exact time not needed

In [33]:
df.sample(5)

,DATE,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,ROAD_CONDITION,ACCLASS,INJURY,FATAL_NO,HOOD_158,NEIGHBOURHOOD_158,DOW,HOUR,YEAR,MONTH,SEVERE_COLLISION
15760,2019-07-26,THE QUEENSWAY,VIBE RD,Etobicoke York,43.621604,-79.523433,Clear,Daylight,Dry,Non-Fatal Injury,Minimal,0.0,159,Etobicoke City Centre,Friday,14,2019,7,0
18739,2023-09-13,ISLINGTON AV,SPRINGBROOK GS,NaN,43.640681,-79.521702,Clear,Dark,Dry,Non-Fatal Injury,Minimal,0.0,16,Stonegate-Queensway,Wednesday,18,2023,9,0
346,2006-04-26,KINGSTON RD,WARDEN AVE,Scarborough,43.687745,-79.270890,Clear,Daylight,Dry,Non-Fatal Injury,Minor,0.0,122,Birchcliffe-Cliffside,Wednesday,9,2006,4,0
6214,2010-08-21,WARDEN AVE,HOLLIS AVE,Scarborough,43.692845,-79.272990,Clear,Dark,Dry,Non-Fatal Injury,Major,0.0,122,Birchcliffe-Cliffside,Saturday,3,2010,8,1
2407,2007-08-18,MADELAINE AVE,DANFORTH AVE,Scarborough,43.692045,-79.284490,Clear,Daylight,Dry,Non-Fatal Injury,Major,0.0,121,Oakridge,Saturday,13,2007,8,1


In [34]:
def categorize_hour(hour):
    if 0 <= hour < 4:
        return 'Late Night'
    elif 4 <= hour < 8:
        return 'Early Morning'
    elif 8 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 16:
        return 'Afternoon'
    elif 16 <= hour < 20:
        return 'Evening Rush'
    else:
        return 'Night'

In [35]:
def categorize_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

In [36]:
df['TIME_OF_DAY'] = df['HOUR'].apply(categorize_hour)
df['SEASON'] = df['MONTH'].apply(categorize_season)

In [37]:
df['ROAD_CONDITION'] = df['ROAD_CONDITION'].replace(['', ' ', 'nan', 'NaN'], np.nan)
df.loc[df['ROAD_CONDITION'].isna() & (df['SEVERE_COLLISION'] == 1), 'ROAD_CONDITION'] = 'Wet'
df.loc[df['ROAD_CONDITION'].isna() & (df['SEVERE_COLLISION'] == 0), 'ROAD_CONDITION'] = 'Dry'

In [38]:
df['LIGHT'] = df['LIGHT'].replace(['', ' ', 'nan', 'NaN'], np.nan)
df.loc[df['LIGHT'].isna() & (df['SEVERE_COLLISION'] == 1), 'LIGHT'] = 'Dark'
df.loc[df['LIGHT'].isna() & (df['SEVERE_COLLISION'] == 0), 'LIGHT'] = 'Daylight'

In [39]:
df['VISIBILITY'] = df['VISIBILITY'].replace(['', ' ', 'nan', 'NaN'], np.nan)
df.loc[df['VISIBILITY'].isna() & (df['SEVERE_COLLISION'] == 1), 'VISIBILITY'] = 'Rain'
df.loc[df['VISIBILITY'].isna() & (df['SEVERE_COLLISION'] == 0), 'VISIBILITY'] = 'Clear'

In [40]:
df.shape

(18957, 21)

In [41]:
df.to_csv('ksi_collisions.csv', index=False)